In [1]:
!pip install outlines
!pip install context_cite

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached spacy-3.8.4-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached murmurhash-1.0.12-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached cymem-2.0.11-cp311-cp311-win_amd64.whl.metadata (8.8 kB)
  Using cached preshed-3.0.9-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
  Using cached thinc-8.3.4-cp311-cp311-win_amd64.whl.metadata (15 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached srsly-2.5.1-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-an

## Imports

In [2]:
%load_ext autoreload
%autoreload 2

# Add the path to the parent directory to sys
import sys, os

# If current directory is called 'notebooks', chdir to the parent
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('../')
    
sys.path.append('attribution')

from torch.utils.data import DataLoader

import pandas as pd
from constants import ModelNames, DatasetNames, LANGUAGE_MAPPING
from model_utils import Model 
from dataset_utils import GSMDataset, PaddingCollator, is_correct_gsm, extract_answer_gsm
from context_cite import ContextCiter
from tqdm.notebook import tqdm
from attribution.cleaning import ResponseProcessor

import warnings

# Filter specific warning categories
warnings.filterwarnings("ignore", category=UserWarning)  # For general user warnings
warnings.filterwarnings("ignore", category=FutureWarning)  # For deprecation warnings

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Steps:
 1. read from "analysis_{model_name}"
 2. pass in model_generated_steps and query
 3. Check if there answer matches with our answer (I think it might be worthwile to also check wrong answers.)
 4. If yes, then use cc.getattribution() to attribution [contextCite](https://github.com/MadryLab/context-cite)
 5. Save the np.array to the respective row of the "analysis_{model_name}" set

In [7]:
context_model = Model(ModelNames.QwenInstruct)

# Unlike RAG, the context follows the query
prompt_template = '{query}\n{context}'

model_responses = pd.read_csv(f'results/processed/mgsm_fr_COT_constrained_Qwen2.5-1.5B-Instruct_results.csv')
model_responses.head()

Device set to use cuda:0


,question,actual_answer,model_gen_steps,model_gen_answer,model_answer_str
0,Question : Roger a 5 balles de tennis. Il achè...,18,Step-by-Step Answer:\n1 Janet pond 16 œufs pa...,18.0,La réponse est 18.<
1,Question : Roger a 5 balles de tennis. Il achè...,3,Step-by-Step Answer:\n1 Une robe nécessite 2 ...,3.0,La réponse est 3.<
2,Question : Roger a 5 balles de tennis. Il achè...,70000,Step-by-Step Answer:\n1 Le prix initial de la...,195000.0,La réponse est 195000.<
3,Question : Roger a 5 balles de tennis. Il achè...,540,Step-by-Step Answer:\n1 Jacques fait 3 sprint...,540.0,La réponse est 540.<
4,Question : Roger a 5 balles de tennis. Il achè...,20,Step-by-Step Answer:\n1 Wendi donne 15 bols d...,20.0,La réponse est 20.<


In [ ]:
cite_df = pd.DataFrame()

# Get length of model_responses
len_responses = len(model_responses)

# initialize a progress bar
pbar = tqdm(total=len_responses)
error_counter = 0

# Iterate over the rows of the DataFrame
for index, row in model_responses.iterrows():
    pbar.update(1)
    context = row['model_gen_steps']
    query = row['question']
    answer_string = row['model_answer_str']
    
    # Abstain from pre-train because it creates a new model each time
    # Constructor is needed due to processing during initialization
    cc = ContextCiter(context_model.model, context_model.tokenizer, context, query, prompt_template=prompt_template)
    
    # We want to use precomputed answers
    # See https://github.com/MadryLab/context-cite/issues/4
    _, prompt = cc._get_prompt_ids(return_prompt=True)
    cc._cache["output"] = prompt + answer_string
    
    # This returns an importance for each line in the context
    # The progress bar is annoying
    line_importance = cc.get_attributions(as_dataframe=False, verbose=False)
    
    # Get each line and importance and add to df
    lines = context.split('\n')
    
    # If number of lines and importance values do not match, raise an error
    if len(lines) != len(line_importance):
        print(f"Number of lines ({len(lines)}) and importance values ({len(line_importance)}) do not match in example {index} Skipping...")
        error_counter += 1
        continue
    
    # Create a temporary DataFrame with sample_index to identify which example each line belongs to
    temp_df = pd.DataFrame({
        'sample_index': index,  # Use the DataFrame index as sample index
        'line': lines,
        'importance': line_importance
    })
    
    cite_df = pd.concat([cite_df, temp_df], ignore_index=True)
    
pbar.close()
print(f"Number of errors: {error_counter}")

  0%|          | 0/250 [00:00<?, ?it/s]

Number of errors: 0


In [10]:
# Store results as JSON
import json
import numpy as np

# Create a list to store one dictionary per question
result_list = []

# Custom JSON encoder to handle NumPy types
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (np.integer, np.int64)):
            return int(obj)
        elif isinstance(obj, (np.floating, np.float64)):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return super().default(obj)

for sample_index, group in cite_df.groupby('sample_index'):
    original_row = model_responses.iloc[sample_index]
    
    # Create a dictionary for this sample
    sample_dict = {
        'sample_index': sample_index,  # No need to manually convert
        'question': original_row['question'],
        'actual_answer': int(float(original_row['actual_answer'])),
        'model_gen_answer': int(float(original_row['model_gen_answer'])),
        'model_answer_str': original_row['model_answer_str'],
        'lines_and_importance': [
            {'text': row['line'], 'importance': row['importance']} 
            for _, row in group.iterrows()
        ]  # No need to manually convert
    }
    
    # Add this dictionary to our results list
    result_list.append(sample_dict)

# Save as JSON file with proper formatting and custom encoder
with open('results/contextcite_fr_QwenInstruct_COT.json', 'w', encoding='utf-8') as f:
    json.dump(result_list, f, ensure_ascii=False, indent=2, cls=NumpyEncoder)